In [35]:
HF_EMBBED_ID = 'mistralai/Mistral-7B-v0.1'
HF_MODEL_ID = 'LACAI/DialoGPT-small-SGD'
USE_HF_MODEL = True

In [36]:
import dotenv
import getpass
dotenv.load_dotenv()
ENV = dotenv.dotenv_values()

In [37]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface_hub import HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma

api_key= ENV['HUGGINGFACEHUB_API_TOKEN'] if ENV['HUGGINGFACEHUB_API_TOKEN'] else getpass.getpass("Hugging Face API Access Token: "),

embedding = HuggingFaceHubEmbeddings() if USE_HF_MODEL else OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id=HF_MODEL_ID, model_kwargs={"temperature": 0.5}
) if USE_HF_MODEL else ChatOpenAI()
retriever = vectorstore.as_retriever()

In [40]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

template = """
You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
If the question is not related to the context, politely respond that you are teached to only answer questions that are related to the context.
If you don't know the answer, just say you don't know. DO NOT try to make up an answer. Try to make the title for every answer if it is possible. Answer in markdown.
Use as much detail as possible when responding and try to make answer in markdown format as much as possible.

{context}

{chat_history}
Question: {question}
Answer in markdown format:
  """

prompt = PromptTemplate(input_variables=["chat_history", "question", "context"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory,combine_docs_chain_kwargs={"prompt": prompt})

In [41]:
qa({'question':"What is the mathematics"})

{'question': 'What is the mathematics', 'chat_history': '', 'answer': '!!'}